<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.

In [ ]:
import os
import json
import datetime
import subprocess

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}"
cert_manager_yaml = "https://github.com/jetstack/cert-manager/releases/download/v1.14.1/cert-manager.yaml"

def run_command(command):  
    try:  
        output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        return True, output.decode("utf-8")
    except subprocess.CalledProcessError as e:
        return False, e.output.decode("utf-8")

def log(stdout, name, action):
    if stdout.n.startswith("ERROR"):
        print("👎🏻 ", name, " was NOT ", action, ": ", stdout)
    else:
        print("👍🏻 ", name, " was ", action, " ⌚ ", datetime.datetime.now().time())

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} -o json  
deployment = json.loads(deployment_stdout.n)
for resource in deployment.get("properties").get("outputResources"):
    resource_id = resource.get("id")
    try:
        query = "\"{type:type, name:name, location:location}\""
        resource_stdout = ! az resource show --id {resource_id} --query {query} -o json
        resource = json.loads(resource_stdout.n)
        resource_name = resource.get("name")
        resource_type = resource.get("type")
        if resource_type == "Microsoft.CognitiveServices/accounts":
            resource_location = "\"" + resource.get("location") + "\""
            delete_stdout = ! az cognitiveservices account delete -g {resource_group_name} -n {resource_name}
            log(delete_stdout, resource_name, "deleted")
            delete_stdout = ! az cognitiveservices account purge -g {resource_group_name} -n {resource_name} -l {resource_location}
            log(delete_stdout, resource_name, "purged")
        elif resource_type == "Microsoft.ApiManagement/service":
            resource_location = "\"" + resource.get("location") + "\""
            delete_stdout = ! az apim delete -n {resource_name} -g {resource_group_name} -y
            log(delete_stdout, resource_name, "deleted")
            delete_stdout = ! az apim deletedservice purge --service-name {resource_name} --location {resource_location}
            log(delete_stdout, resource_name, "purged")
        elif resource_type == "Microsoft.ContainerService/managedClusters":
            delete_stdout = ! az aks delete --name {resource_name} --resource-group {resource_group_name} --yes --no-wait
            log(delete_stdout, resource_name, "deleted")
    except:
        print("✌🏻 ", resource_id, " ignored")
        
# Clean up Kubernetes resources
kubectl_delete_resources = [
    "kubectl delete namespace azureserviceoperator-system",
    "kubectl delete namespace cert-manager",
    "kubectl delete -f {cert_manager_yaml}",
    "kubectl delete -f rg.yaml"  # Assuming you have a resource group YAML file
]
  
for cmd in kubectl_delete_resources:  
    delete_stdout = ! {cmd}  
    log(delete_stdout, cmd, "executed") 
    
# Delete the azure service operator
sp_name = "azure-service-operator"  
sp_delete_stdout = ! az ad sp delete --id http://{sp_name}  
        
delete_stdout = ! az group delete --name {resource_group_name} -y
log(delete_stdout, resource_group_name, "deleted")